In [1]:
import os
import pandas as pd
import numpy as np

In [2]:
# read the data
df=pd.read_csv('/Users/web/Library/CloudStorage/Dropbox/NTU_course/Senior1/ML_HT/code/final_regular/JSH/train_data_add_nearby/500101008.csv', header=None)
df

,0,1,2,3,4,5,6,7
0,500101008,20231002,1,17,1,25.02112,121.53407,16
1,500101008,20231002,2,17,1,25.02112,121.53407,16
2,500101008,20231002,3,17,1,25.02112,121.53407,16
3,500101008,20231002,4,17,1,25.02112,121.53407,16
4,500101008,20231002,5,17,1,25.02112,121.53407,16
...,...,...,...,...,...,...,...,...
60675,500101008,20231123,1435,17,1,25.02112,121.53407,6
60676,500101008,20231123,1436,17,1,25.02112,121.53407,6
60677,500101008,20231123,1437,17,1,25.02112,121.53407,6
60678,500101008,20231123,1438,17,1,25.02112,121.53407,6


In [3]:
# name the columns
df.columns = ["stationID", "date", "minute of day", "tot", "act", "lng", "lat", "bikes"]
df

,stationID,date,minute of day,tot,act,lng,lat,bikes
0,500101008,20231002,1,17,1,25.02112,121.53407,16
1,500101008,20231002,2,17,1,25.02112,121.53407,16
2,500101008,20231002,3,17,1,25.02112,121.53407,16
3,500101008,20231002,4,17,1,25.02112,121.53407,16
4,500101008,20231002,5,17,1,25.02112,121.53407,16
...,...,...,...,...,...,...,...,...
60675,500101008,20231123,1435,17,1,25.02112,121.53407,6
60676,500101008,20231123,1436,17,1,25.02112,121.53407,6
60677,500101008,20231123,1437,17,1,25.02112,121.53407,6
60678,500101008,20231123,1438,17,1,25.02112,121.53407,6


In [4]:
# make date_time format correct
df[['hr','minutes']] = df['minute of day'].apply(lambda x: divmod(x, 60)).apply(pd.Series)

# Combine 'date' and 'time' columns into a single datetime column
df['datetime'] = pd.to_datetime( df['date'].astype(str) + ' ' + df['hr'].astype(str) + ":" + df['minutes'].astype(str),  format='%Y%m%d %H:%M' )
df.set_index('datetime',inplace=True)
tot = df['tot'][0]


In [5]:
bike_number = df["bikes"]
bike_number.head()


datetime
2023-10-02 00:01:00    16
2023-10-02 00:02:00    16
2023-10-02 00:03:00    16
2023-10-02 00:04:00    16
2023-10-02 00:05:00    16
Name: bikes, dtype: int64

In [6]:
# normalize the data
from sklearn.preprocessing import MinMaxScaler
bike_number_2d = bike_number.values.reshape(-1, 1)
normalized_bike_number = MinMaxScaler().fit_transform( bike_number_2d )

pd.DataFrame(normalized_bike_number)
df['normalized_bikes'] = normalized_bike_number
normalized_bike_number = df['normalized_bikes']
df.head()

,stationID,date,minute of day,tot,act,lng,lat,bikes,hr,minutes,normalized_bikes
datetime,,,,,,,,,,,
2023-10-02 00:01:00,500101008,20231002,1,17,1,25.02112,121.53407,16,0,1,0.941176
2023-10-02 00:02:00,500101008,20231002,2,17,1,25.02112,121.53407,16,0,2,0.941176
2023-10-02 00:03:00,500101008,20231002,3,17,1,25.02112,121.53407,16,0,3,0.941176
2023-10-02 00:04:00,500101008,20231002,4,17,1,25.02112,121.53407,16,0,4,0.941176
2023-10-02 00:05:00,500101008,20231002,5,17,1,25.02112,121.53407,16,0,5,0.941176


In [7]:
# function to make the format of data like below
# [[[1], [2], [3], [4], [5]]] [6]
# [[[2], [3], [4], [5], [6]]] [7]
# [[[3], [4], [5], [6], [7]]] [8]

def df_to_X_y(df, tot, window_size=5):
  df_as_np = df.to_numpy()
  X = []
  y = []
  for i in range(len(df_as_np)-window_size):
    row = [[a] for a in df_as_np[i:i+window_size]]
    X.append(row)
    label = df_as_np[i+window_size]
    y.append(label)
  return np.array(X), np.array(y)

In [8]:
WINDOW_SIZE = 100
X1, y1 = df_to_X_y(normalized_bike_number, tot, WINDOW_SIZE)
X1.shape, y1.shape

((60580, 100, 1), (60580,))

In [9]:
X1

array([[[0.94117647],
        [0.94117647],
        [0.94117647],
        ...,
        [0.94117647],
        [0.94117647],
        [0.94117647]],

       [[0.94117647],
        [0.94117647],
        [0.94117647],
        ...,
        [0.94117647],
        [0.94117647],
        [0.94117647]],

       [[0.94117647],
        [0.94117647],
        [0.94117647],
        ...,
        [0.94117647],
        [0.94117647],
        [0.94117647]],

       ...,

       [[0.64705882],
        [0.64705882],
        [0.64705882],
        ...,
        [0.35294118],
        [0.35294118],
        [0.35294118]],

       [[0.64705882],
        [0.64705882],
        [0.64705882],
        ...,
        [0.35294118],
        [0.35294118],
        [0.35294118]],

       [[0.64705882],
        [0.64705882],
        [0.70588235],
        ...,
        [0.35294118],
        [0.35294118],
        [0.35294118]]])

In [10]:
X_train1, y_train1 = X1[:50000], y1[:50000]
X_val1, y_val1 = X1[50000:55000], y1[50000:55000]
X_test1, y_test1 = X1[55000:], y1[55000:]
# X_train1.shape, y_train1.shape, X_val1.shape, y_val1.shape, X_test1.shape, y_test1.shape
X1[-35]

array([[0.94117647],
       [0.82352941],
       [0.76470588],
       [0.82352941],
       [0.70588235],
       [0.70588235],
       [0.70588235],
       [0.64705882],
       [0.58823529],
       [0.52941176],
       [0.52941176],
       [0.52941176],
       [0.52941176],
       [0.52941176],
       [0.52941176],
       [0.52941176],
       [0.52941176],
       [0.52941176],
       [0.47058824],
       [0.47058824],
       [0.47058824],
       [0.47058824],
       [0.47058824],
       [0.47058824],
       [0.47058824],
       [0.52941176],
       [0.52941176],
       [0.64705882],
       [0.64705882],
       [0.64705882],
       [0.64705882],
       [0.64705882],
       [0.64705882],
       [0.64705882],
       [0.64705882],
       [0.64705882],
       [0.70588235],
       [0.70588235],
       [0.70588235],
       [0.70588235],
       [0.70588235],
       [0.64705882],
       [0.64705882],
       [0.64705882],
       [0.64705882],
       [0.58823529],
       [0.58823529],
       [0.588

## LSTM model

In [11]:
from keras.models import Sequential
from keras.layers import *
from keras.callbacks import ModelCheckpoint
from keras.losses import MeanSquaredError
from keras.metrics import RootMeanSquaredError
from keras.optimizers.legacy import Adam


model1 = Sequential()
model1.add(InputLayer((WINDOW_SIZE, 1)))
model1.add(LSTM(500))
model1.add(Dense(8, 'relu'))
model1.add(Dense(1, 'linear'))

model1.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 500)               1004000   
                                                                 
 dense (Dense)               (None, 8)                 4008      
                                                                 
 dense_1 (Dense)             (None, 1)                 9         
                                                                 
Total params: 1008017 (3.85 MB)
Trainable params: 1008017 (3.85 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [12]:
cp1 = ModelCheckpoint('model1/', save_best_only=True)
model1.compile(loss='mean_absolute_error', optimizer=Adam(learning_rate=0.00001), metrics=[RootMeanSquaredError()])

In [13]:
model1.fit(X_train1, y_train1, validation_data=(X_val1, y_val1), epochs=5, callbacks=[cp1])

Epoch 1/5
1563/1563 [==============================] - ETA: 0s - loss: 0.0551 - root_mean_squared_error: 0.1275INFO:tensorflow:Assets written to: model1/assets


INFO:tensorflow:Assets written to: model1/assets


1563/1563 [==============================] - 287s 183ms/step - loss: 0.0551 - root_mean_squared_error: 0.1275 - val_loss: 0.0306 - val_root_mean_squared_error: 0.0590
Epoch 2/5
1130/1563 [====================>.........] - ETA: 1:13 - loss: 0.0273 - root_mean_squared_error: 0.0572

KeyboardInterrupt: 

In [ ]:
# prediction on training data
train_predictions = model1.predict(X_train1).flatten()
train_results = pd.DataFrame(data={'Train Predictions':train_predictions, 'Actuals':y_train1})
train_results.tail(50)

In [ ]:
import matplotlib.pyplot as plt
plt.plot(train_results['Train Predictions'][:10000])
plt.plot(train_results['Actuals'][:10000])

In [ ]:
# prediction on validation data
val_predictions = model1.predict(X_val1).flatten()
val_results = pd.DataFrame(data={'Val Predictions':val_predictions, 'Actuals':y_val1})
val_results

In [ ]:
# prediction on validation
plt.plot(val_results['Val Predictions'][:500])
plt.plot(val_results['Actuals'][:500])

In [ ]:
# print(len(X_test1))
# print( type(model1.predict(X_test1)) )
test_predictions = model1.predict(X_test1).flatten()
test_results = pd.DataFrame(data={'Test Predictions':test_predictions, 'Actuals':y_test1})
# test_results
# print(X_test1)

In [ ]:
plt.plot(test_results['Test Predictions'][:10000], label = "predict")
plt.plot(test_results['Actuals'][:10000], label = "True")
plt.legend()

In [ ]:
np.set_printoptions(threshold=np.inf)
# Generate forecasts
forecast_inputs = X_test1[-1:]  # Use the last 'look_back' values as input
# print(forecast_inputs)
# forecast_inputs = forecast_inputs.reshape(1, 1, 1)
# print(forecast_inputs)

forecast_results = []
for i in range(3000):  # forecast the next 30 time steps
    # print(forecast_inputs)
    prediction = model1.predict(forecast_inputs, verbose=0)
    forecast_results.append(prediction[0, 0])
    
    # update forecast_inputs
    forecast_inputs = np.array([ forecast_inputs[0][1:] ])
    forecast_inputs = np.concatenate( [ forecast_inputs, np.array([ prediction ]) ], axis=1 )
    # print(forecast_inputs)



print(forecast_results)



In [ ]:
# plt.plot(data, label='Original Data')
# plt.plot(np.arange(len(data), len(data) + len(forecast)), forecast, label='Forecast')
plt.plot(forecast_results, label = "forecast")
plt.legend()